In [1]:
from qiskit import QuantumCircuit , execute, Aer 
from numpy import pi 

In [2]:
def test_loader(file_path):
    # make it faster ...
    arz, arx, ary = [], [], []
    
    with open(file_path, "r") as file:
        for row_id, row in enumerate(file):
            if row_id == 0:
                state = row[0]
            else:
                elements = [int(x) for x in row[:-1].split(",")]
                if row_id == 1:
                    arz = elements
                elif row_id == 2:
                    arx = elements 
                else:
                    ary = elements

    return state, arz, arx, ary

In [3]:
INPUT_PATH = "tests/inputs/"
OUTPUT_PATH = "tests/outputs/"

In [4]:
def initialize(state, qc):
    if state == "1":
        qc.x(0)
    elif state == "+":
        qc.h(0)
    elif state == "-":
        qc.x(0)
        qc.h(0)
    elif state == "r":
        qc.h(0)
        qc.s(0)
    elif state == "l":
        qc.h(0)
        qc.rz(-pi*0.5, 0)

In [5]:
backend = Aer.get_backend('qasm_simulator')
def brute_bloch_ops(state, arz, arx, ary):
    total = 0
    qc_list = []
    for z in arz:
        for x in arx:
            for y in ary:
                qc = QuantumCircuit(1)
                initialize(state, qc)
                
                qc.rz(z*pi/2,0)
                qc.rx(x*pi/2,0)
                qc.ry(y*pi/2,0)
                
                qc.measure_all()
                qc_list.append(qc)
                
    
    counts = execute(qc_list, backend, shots = 20).result().get_counts()
    if isinstance(counts,dict):
        counts = [counts]
    
    for count in counts:
        if len(count) == 1 and "0" in count:
            total+=1
    
    return total

In [13]:
def get_total_bloch_ops(state, arz, arx, ary):
    # optimized version
    def handle_rot(angle):
        return angle % 4 
        
    ARZ, ARX, ARY = {}, {}, {}
    
    for i in range(4):
        ARZ[i] = ARX[i] = ARY[i] = 0 
        
    for i in arz:
        net_z = handle_rot(i)
        ARZ[net_z] += 1
        
    for j in arx:
        net_x = handle_rot(j)
        ARX[net_x] += 1
        
    for k in ary:
        net_y = handle_rot(k)
        ARY[net_y] += 1
        
    # now evaluate answer 
    product_dict = {}
    
    for i in range(4):
        for j in range(4):
            for k in range(4):
                product_dict[(i,j,k)] = ARZ[i]*ARX[j]*ARY[k]
                
    total = 0 
    
    
    if state == '0':
        # every z rotation is okay 
        
        total+= len(arz) * ARX[0] * ARY[0]
        total+= len(arz) * ARX[2] * ARY[2]
        
    elif state == '1':
        
        # here, if x as 0, then y as 2 
        
        # if x as 2, y as 0 
        total+= len(arz) * ARX[0] * ARY[2]
        total+= len(arz) * ARX[2] * ARY[0]
        
    elif state == '+':
        
        total += ARZ[1]*ARX[1]*ARY[0]
        total += ARZ[1]*ARX[3]*ARY[2]
        
        total += ARZ[3]*ARX[1]*ARY[2]
        total += ARZ[3]*ARX[3]*ARY[0]
        
        total += ARZ[0]*len(arx)*ARY[3]
        total += ARZ[2]*len(arx)*ARY[1]
        
        
    elif state == '-':
        
        total += product_dict[(1,1,2)] 
        total += product_dict[(1,3,0)]
        
        total += product_dict[(3,1,0)]
        total += product_dict[(3,3,2)]
        
        total += ARZ[0]*len(arx)*ARY[1]
        total += ARZ[2]*len(arx)*ARY[3]
            
    elif state == 'r':
        
        total += product_dict[(0,1,0)] + product_dict[(0,3,2)]
        
        total += product_dict[(2,1,2)] + product_dict[(2,3,0)]
        
        total += ARZ[3]*len(arx)*ARY[3]
        total += ARZ[1]*len(arx)*ARY[1]
        
    else:
        # 'l'
        
        total += product_dict[(0,1,2)] + product_dict[(0,3,0)]
        
        total += product_dict[(2,1,0)] + product_dict[(2,3,2)]
        
        total += ARZ[1]*len(arx)*ARY[3]
        total += ARZ[3]*len(arx)*ARY[1]
        
    return total 
    

In [7]:
def output_saver(file_path, total_bloch_ops):
    with open(file_path, 'w') as file:
        file.write(str(total_bloch_ops))
        file.write("\n")

In [8]:
class answer_gen_2:
    ip_task_path = INPUT_PATH + "task-2-"
    op_task_path = OUTPUT_PATH + "task-2-"
    total_tests = 10
    
    @classmethod
    def generate_answer(cls):
        for test in range(answer_gen_2.total_tests):
            ip_test_path = cls.ip_task_path + str(test) + ".txt"
            state, arz, arx, ary = test_loader(ip_test_path)
            print("Input test : ", ip_test_path)
            total_bloch_ops = get_total_bloch_ops(state, arz, arx, ary)
            brute = brute_bloch_ops(state, arz, arx, ary)
            
            if(total_bloch_ops != brute):
                print("Wrong for test ",ip_test_path)
                print("State : ", state)
                print("Brute :", brute)
                print("Optimized :", total_bloch_ops)
            else:
                print("Correct!")
            # store to the outputs 
            op_test_path = cls.op_task_path + str(test) + ".txt"
            
            output_saver(op_test_path , total_bloch_ops)

In [ ]:
answer_gen_2.generate_answer()

In [11]:
class answer_gen_3:
    ip_task_path = INPUT_PATH + "task-3-"
    op_task_path = OUTPUT_PATH + "task-3-"
    total_tests = 10
    
    @classmethod
    def generate_answer(cls):
        for test in range(answer_gen_3.total_tests):
            ip_test_path = cls.ip_task_path + str(test) + ".txt"
            state, arz, arx, ary = test_loader(ip_test_path)
            print("Input test : ", ip_test_path)
            total_bloch_ops = get_total_bloch_ops(state, arz, arx, ary)
#             brute = brute_bloch_ops(state, arz, arx, ary)
            
#             if(total_bloch_ops != brute):
#                 print("Wrong for test ",ip_test_path)
#                 print("State : ", state)
#                 print("Brute :", brute)
#                 print("Optimized :", total_bloch_ops)
#             else:
#                 print("Correct!")
            # store to the outputs 
            op_test_path = cls.op_task_path + str(test) + ".txt"
            
            output_saver(op_test_path , total_bloch_ops)

In [ ]:
answer_gen_3.generate_answer()